In [ ]:
!pip install yfinance --upgrade --no-cache-dir
!pip install yahoo_fin --upgrade --no-cache-dir

     |████████████████████████████████| 6.3 MB 10.9 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=fb6f2cc6132fb63d9abe0b7807cf0259296fb5da3ccd0eac0c45d778aa126f4f
  Stored in directory: /tmp/pip-ephem-wheel-cache-bxlz85sy/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 81 kB 9.3 MB/s 
     |████████████████████████████████| 83 kB 46.5 MB/s 
     |████████████████████████████████| 103 kB 62.3 MB/s 
     |████████████████████████████████| 127 kB 73.0 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=df06a569d22ca1ac55ff9c6d37af6291ca1fa0eb2517d7eca013912abc55bb29
  Stored in directory: /tmp/pip-ephem-wheel-cache-jj6w7hyc/wheels/ed/f7/62/50ab6c9a0b5567

In [ ]:
# ESTRATEGIA 0 DOLLAR COST AVERAGE
from datetime import date, timedelta
import matplotlib.pyplot as plt
import yfinance as yf  
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from datetime import datetime

import pandas as pd
import numpy as np
stock0 = "QQQ"
dias = 40
v0 = 2000
fecha_i = date(2000,1,1)
#fecha_f = date.today()
fecha_f = date(2021,8,31)
meses =  (fecha_f.year - fecha_i.year) * 12 + fecha_f.month - fecha_i.month
DF0 = yf.download(stock0,fecha_i,fecha_f)
DF0['Date'] = DF0.index
DateCompras=[]
ValorCompras=[]
NumStocks = []
Compras=[]
Invertido=[]

compras0 = 0
TotalInvertido = 0
j=-1
nstocks0=0
nstocks1=0
max = -1
for i in DF0.index:
  j+=1
  if (DF0['Open'][j] > max) : 
    max = DF0['Open'][j]

  if (j%dias == 0 ): 
      nstocks0 += round(v0 /pd.to_numeric(DF0['Open'])[j],1)
      NumStocks.append(nstocks0)
      compras0 += 1
      DateCompras.append(DF0['Date'][j])
      ValorCompras.append(pd.to_numeric(DF0['Open'])[j])
      TotalInvertido += round(v0,1)
      Invertido.append(TotalInvertido)
      #print(DF0['Date'][j], "Compra=",round(v0*(frac),1),"frac=",round(frac,2),"Open=",round(pd.to_numeric(DF0['Open'])[j],1),"max=",round(max,1) )
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=DF0['Date'], y=DF0['Open'], name=stock0),
        secondary_y=False)
fig.add_trace(go.Scatter(x=DateCompras, y=ValorCompras, name="Compras",mode='markers',marker_symbol='circle'),
        secondary_y=False)
fig.add_trace(go.Scatter(x=DateCompras, y=Invertido,mode='markers', name="Invertido",marker_symbol='circle'),
        secondary_y=True,
    )
fig.update_layout(title_text="Dollar Cost Average with "+stock0,title_x=.4,title_y=.9)
fig.update_xaxes(title_text="Fechas")

fig.update_yaxes(title_text="Valor de Accion de "+stock0, secondary_y=False)
fig.update_yaxes(title_text="Total Invertido", secondary_y=True)
fig.update_traces(marker=dict(size=8,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
#fig.update_layout(yaxis_type="log")
fig.show()
ValorTotal = round(DF0['Close'][-1]*NumStocks[-1],1)
print("Numero de meses = ",meses,"Total compras:", compras0)
rend = round(ValorTotal/TotalInvertido,2)
print( "Valor Final "+stock0+" = ",round(DF0['Close'][-1],1),"No. Stocks: ", round(NumStocks[-1],1))
print("Total invertido:",round(TotalInvertido,1),"Valor total:",ValorTotal)
print("Rendimiento:", rend,"Promedio mensual = ", round(TotalInvertido/meses,1))

[*********************100%***********************]  1 of 1 completed


Numero de meses =  259 Total compras: 137
Valor Final QQQ =  380.3 No. Stocks:  4762.8
Total invertido: 274000 Valor total: 1811102.4
Rendimiento: 6.61 Promedio mensual =  1057.9


In [ ]:
# VARIANTE 1 A DOLLAR COST AVERAGE 
# COMPRAR SI EL PRECIO EN DIAS POSTERIORES A COMPRA PERIODICA ES MAS BAJO
from datetime import date, timedelta
import matplotlib.pyplot as plt
import yfinance as yf  
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

import pandas as pd
import numpy as np
stock0 = "QQQ"
dias = 40
v0 = 250
v1 = 250
fecha_i = date(2000,1,1)
#fecha_f = date.today()
fecha_f = date(2021,8,31)
meses =  (fecha_f.year - fecha_i.year) * 12 + fecha_f.month - fecha_i.month
DF0 = yf.download(stock0,fecha_i,fecha_f)
DF0['Date'] = DF0.index
DateCompras=[]
DateCompras0=[]
DateCompras1=[]
Invertido0=[]
Invertido1=[]

ValorCompras0=[]
ValorCompras1=[]
NumStocks = []

Compras=[]
compras0 = 0
TotalInvertido = 0
j=-1
nstocks0=0
nstocks1=0
max = -1

for i in DF0.index:
  j+=1
  if (j==0):
    cambio = 0
    last_value = pd.to_numeric(DF0['Open'])[0]
  else:
    cambio = (pd.to_numeric(DF0['Open'])[j] - last_value )/last_value
  if (j%dias != 0 and pd.to_numeric(DF0['Open'])[j]< last_value) : 
    compras0 += 1
    TotalInvertido += round(v1,1)
    Invertido1.append(TotalInvertido)

    #print (DF0['Date'][j],j,cambio, compras0,TotalInvertido)
    nstocks0 += v1 /pd.to_numeric(DF0['Open'])[j]
    NumStocks.append(nstocks0)
    DateCompras1.append(DF0['Date'][j])
    ValorCompras1.append(pd.to_numeric(DF0['Open'])[j])
    
    #last_value = pd.to_numeric(DF0['Open'])[j]

  if (j%dias == 0): 
    compras0 += 1
    TotalInvertido += round(v0,1)

    #print(DF0['Date'][j],j,compras0,TotalInvertido)
    nstocks0 += v0 /pd.to_numeric(DF0['Open'])[j]
    NumStocks.append(nstocks0)
    compras0 += 1
    DateCompras0.append(DF0['Date'][j])
    Invertido0.append(TotalInvertido)

    ValorCompras0.append(pd.to_numeric(DF0['Open'])[j])
    last_value = pd.to_numeric(DF0['Open'])[j]
    #print(DF0['Date'][j], "Compra=",round(v0*(frac),1),"frac=",round(frac,2),"Open=",round(pd.to_numeric(DF0['Open'])[j],1),"max=",round(max,1) )
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=DF0['Date'], y=DF0['Open'], name=stock0),
        secondary_y=False)
fig.add_trace(go.Scatter(x=DateCompras0, y=ValorCompras0, name="Compras",mode='markers',marker_symbol='circle'),
        secondary_y=False)
fig.add_trace(go.Scatter(x=DateCompras1, y=ValorCompras1, name="Compras Extras",mode='markers',marker_symbol='square'),
        secondary_y=False)
fig.add_trace(go.Scatter(x=DateCompras0, y=Invertido0,mode='markers', name="Invertido",marker_symbol='circle'),
        secondary_y=True)
fig.add_trace(go.Scatter(x=DateCompras1, y=Invertido1,mode='markers', name="Invertido Extra",marker_symbol='square'),
        secondary_y=True)
fig.update_layout(title_text="Variante 1 de Dollar Cost Average with "+stock0+ "<br> Compra Extra si Baja en Dias Posteriores",title_x=.4,title_y=.9)
fig.update_xaxes(title_text="Fechas")

fig.update_yaxes(title_text="Valor de Accion de "+stock0, secondary_y=False)
fig.update_yaxes(title_text="Total Invertido", secondary_y=True)
fig.update_traces(marker=dict(size=8,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
#fig.update_layout(yaxis_type="log")
fig.show()

ValorTotal = round(DF0['Close'][-1]*nstocks0+DF0['Close'][-1]*nstocks1,1)
print("Numero de meses = ",meses,"Total compras:", compras0)
rend = round(ValorTotal/TotalInvertido,2)
print( "Valor Final "+stock0+" = ",round(DF0['Close'][-1],1),"No. Stocks: ", round(NumStocks[-1],1))
print("Total invertido:",round(TotalInvertido,1),"Valor total:",ValorTotal)
print("Rendimiento:", rend,"Promedio mensual = ", round(TotalInvertido/meses,1))

[*********************100%***********************]  1 of 1 completed


Numero de meses =  259 Total compras: 2329
Valor Final QQQ =  380.3 No. Stocks:  10462.5
Total invertido: 548000 Valor total: 3978465.8
Rendimiento: 7.26 Promedio mensual =  2115.8


In [ ]:
# VARIANTE 2 A DOLLAR COST AVERAGE 
# COMPRAR SI EL PRECIO EN DIAS POSTERIORES A COMPRA MENSUAL DA UN PROMEDIO CADA VEZ MAS BAJO

from datetime import date, timedelta
import matplotlib.pyplot as plt
import yfinance as yf  
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

import pandas as pd
import numpy as np
stock0 = "QQQ"
dias = 40
v0 = 250
v1 = 250
fecha_i = date(2000,1,1)
#fecha_f = date.today()
fecha_f = date(2021,8,31)
meses =  (fecha_f.year - fecha_i.year) * 12 + fecha_f.month - fecha_i.month
DF0 = yf.download(stock0,fecha_i,fecha_f)
DF0['Date'] = DF0.index
DateCompras0=[]
DateCompras1=[]
Invertido0=[]
Invertido1=[]
ValorCompras0=[]
ValorCompras1=[]
NumStocks = []



Compras=[]

compras0 = 0
TotalInvertido = 0
j=-1
nstocks0=0
nstocks1=0
max = -1

for i in DF0.index:
  j+=1
  if (j%dias != 0):
    average_mes =  sum(Compras_mes)/sum(Stocks_mes)
    average_mes_new =  (sum(Compras_mes) + v1) /(sum(Stocks_mes) + v1 /pd.to_numeric(DF0['Open'])[j])
  if (j%dias != 0 and average_mes_new < average_mes) : 
    compras0 += 1
    TotalInvertido += round(v1,1)
    Invertido1.append(TotalInvertido)

    #print (DF0['Date'][j],j,cambio, compras0,TotalInvertido)
    nstocks0 += v1 /pd.to_numeric(DF0['Open'])[j]
    NumStocks.append(nstocks0)
    DateCompras1.append(DF0['Date'][j])
    ValorCompras1.append(pd.to_numeric(DF0['Open'])[j])
    Stocks_mes.append(v1 /pd.to_numeric(DF0['Open'])[j])
    Compras_mes.append(v1)
    
    #last_value = pd.to_numeric(DF0['Open'])[j]

  if (j%dias == 0): 
    compras0 += 1
    TotalInvertido += round(v0,1)
    Invertido0.append(TotalInvertido)

    Stocks_mes = [v0 /pd.to_numeric(DF0['Open'])[j]]
    Compras_mes = [v0]
    #print(DF0['Date'][j],j,compras0,TotalInvertido)
    nstocks0 += v0 /pd.to_numeric(DF0['Open'])[j]
    NumStocks.append(nstocks0)
    compras0 += 1
    DateCompras0.append(DF0['Date'][j])
    ValorCompras0.append(pd.to_numeric(DF0['Open'])[j])
    last_value = pd.to_numeric(DF0['Open'])[j]
    #print(DF0['Date'][j], "Compra=",round(v0*(frac),1),"frac=",round(frac,2),"Open=",round(pd.to_numeric(DF0['Open'])[j],1),"max=",round(max,1) )

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=DF0['Date'], y=DF0['Open'], name=stock0),
        secondary_y=False)
fig.add_trace(go.Scatter(x=DateCompras0, y=ValorCompras0, name="Compras",mode='markers',marker_symbol='circle'),
        secondary_y=False)
fig.add_trace(go.Scatter(x=DateCompras1, y=ValorCompras1, name="Compras Extras",mode='markers',marker_symbol='square'),
        secondary_y=False)
fig.add_trace(go.Scatter(x=DateCompras0, y=Invertido0,mode='markers', name="Invertido",marker_symbol='circle'),
        secondary_y=True)
fig.add_trace(go.Scatter(x=DateCompras1, y=Invertido1,mode='markers', name="Invertido Extra",marker_symbol='square'),
        secondary_y=True)
fig.update_layout(title_text="Variante 2 de Dollar Cost Average with "+stock0 + "<br> Compra Extra si Baja Promedio",title_x=.4,title_y=.9)
fig.update_xaxes(title_text="Fechas")

fig.update_yaxes(title_text="Valor de Accion de "+stock0, secondary_y=False)
fig.update_yaxes(title_text="Total Invertido", secondary_y=True)
fig.update_traces(marker=dict(size=8,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
#fig.update_layout(yaxis_type="log")
fig.show()
ValorTotal = round(DF0['Close'][-1]*nstocks0+DF0['Close'][-1]*nstocks1,1)
print("Numero de meses = ",meses,"Total compras:", compras0)
rend = round(ValorTotal/TotalInvertido,2)
print( "Valor Final "+stock0+" = ",round(DF0['Close'][-1],1),"No. Stocks: ", round(NumStocks[-1],1))
print("Total invertido:",round(TotalInvertido,1),"Valor total:",ValorTotal)
print("Rendimiento:", rend,"Promedio mensual = ", round(TotalInvertido/meses,1))

[*********************100%***********************]  1 of 1 completed


Numero de meses =  259 Total compras: 1712
Valor Final QQQ =  380.3 No. Stocks:  7854.0
Total invertido: 393750 Valor total: 2986546.7
Rendimiento: 7.58 Promedio mensual =  1520.3


In [ ]:
# VARIANTE 3 A DOLLAR COST AVERAGE 
# COMPRAR SI EL PRECIO EN DIAS POSTERIORES A COMPRA MENSUAL DA UN PROMEDIO CADA VEZ MAS BAJO UNIENDO PERIODOS

from datetime import date, timedelta
import matplotlib.pyplot as plt
import yfinance as yf  
import plotly.graph_objects as go

from datetime import datetime
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
stock0 = "QQQ"
dias = 40
v0 = 250
v1 = 250
fecha_i = date(2000,1,1)
#fecha_f = date.today()
fecha_f = date(2021,8,31)
meses =  (fecha_f.year - fecha_i.year) * 12 + fecha_f.month - fecha_i.month
DF0 = yf.download(stock0,fecha_i,fecha_f)
DF0['Date'] = DF0.index
DateCompras0=[]
DateCompras1=[]
DateCompras2=[]

Invertido0=[]
Invertido1=[]
Invertido2=[]

ValorCompras0=[]
ValorCompras1=[]
ValorCompras2=[]


Compras=[]

compras0 = 0
TotalInvertido = 0
j=-1
nstocks0=0
nstocks1=0
max = -1

for i in DF0.index:
  j+=1
  if (j%dias != 0):
    average_mes =  sum(Compras_mes)/sum(Stocks_mes)
    average_mes_new =  (sum(Compras_mes) + v1) /(sum(Stocks_mes) + v1 /pd.to_numeric(DF0['Open'])[j])
  if (j%dias != 0 and average_mes_new < average_mes) : 
    compras0 += 1
    TotalInvertido += round(v1,1)
    Invertido1.append(TotalInvertido)
    #print (DF0['Date'][j],j,cambio, compras0,TotalInvertido)
    nstocks0 += v1 /pd.to_numeric(DF0['Open'])[j]
    NumStocks.append(nstocks0)
    DateCompras1.append(DF0['Date'][j])
    ValorCompras1.append(pd.to_numeric(DF0['Open'])[j])
    Stocks_mes.append(v1 /pd.to_numeric(DF0['Open'])[j])
    Compras_mes.append(v1)
    
    #last_value = pd.to_numeric(DF0['Open'])[j]

  if (j%dias == 0): 
    compras0 += 1
    TotalInvertido += round(v0,1)
    
    if (j == 0 or pd.to_numeric(DF0['Open'])[j] > last_value): 
      Stocks_mes = [v0 /pd.to_numeric(DF0['Open'])[j]]
      Compras_mes = [v0]
      nstocks0 += v0 /pd.to_numeric(DF0['Open'])[j]
      NumStocks.append(nstocks0)
      compras0 += 1
      ValorCompras0.append(pd.to_numeric(DF0['Open'])[j])
      DateCompras0.append(DF0['Date'][j])
      Invertido0.append(TotalInvertido)
      last_value = pd.to_numeric(DF0['Open'])[j]
    elif (pd.to_numeric(DF0['Open'])[j] < last_value): 
      nstocks0 += v0 /pd.to_numeric(DF0['Open'])[j]
      NumStocks.append(nstocks0)
      compras0 += 1
      ValorCompras2.append(pd.to_numeric(DF0['Open'])[j])
      DateCompras2.append(DF0['Date'][j])
      Invertido2.append(TotalInvertido)
      last_value = pd.to_numeric(DF0['Open'])[j]
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=DF0['Date'], y=DF0['Open'], name=stock0),
        secondary_y=False)
fig.add_trace(go.Scatter(x=DateCompras0, y=ValorCompras0, name="Compras",mode='markers',marker_symbol='circle'),
        secondary_y=False)
fig.add_trace(go.Scatter(x=DateCompras1, y=ValorCompras1, name="Compras Extras",mode='markers',marker_symbol='square'),
        secondary_y=False)
fig.add_trace(go.Scatter(x=DateCompras2, y=ValorCompras2, name="Compras Cont",mode='markers',marker_symbol='diamond'),
        secondary_y=False)
fig.add_trace(go.Scatter(x=DateCompras0, y=Invertido0,mode='markers', name="Invertido",marker_symbol='circle'),
        secondary_y=True)
fig.add_trace(go.Scatter(x=DateCompras1, y=Invertido1,mode='markers', name="Invertido Extra",marker_symbol='square'),
        secondary_y=True)
fig.add_trace(go.Scatter(x=DateCompras2, y=Invertido2,mode='markers', name="Invertido Cont",marker_symbol='diamond'),
        secondary_y=True)
fig.update_layout(title_text="Variante 3 de Dollar Cost Average with "+stock0 + "<br> Igual a V2 pero Uniendo Periodos",title_x=.4,title_y=.9)
fig.update_xaxes(title_text="Fechas")

fig.update_yaxes(title_text="Valor de Accion de "+stock0, secondary_y=False)
fig.update_yaxes(title_text="Total Invertido", secondary_y=True)
fig.update_traces(marker=dict(size=8,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
#fig.update_layout(yaxis_type="log")
fig.show()
ValorTotal = round(DF0['Close'][-1]*nstocks0+DF0['Close'][-1]*nstocks1,1)
print("Numero de meses = ",meses,"Total compras:", compras0)
rend = round(ValorTotal/TotalInvertido,2)
print( "Valor Final "+stock0+" = ",round(DF0['Close'][-1],1),"No. Stocks: ", round(NumStocks[-1],1))
print("Total invertido:",round(TotalInvertido,1),"Valor total:",ValorTotal)
print("Rendimiento:", rend,"Promedio mensual = ", round(TotalInvertido/meses,1))

[*********************100%***********************]  1 of 1 completed


Numero de meses =  259 Total compras: 2160
Valor Final QQQ =  380.3 No. Stocks:  10608.8
Total invertido: 505750 Valor total: 4034111.7
Rendimiento: 7.98 Promedio mensual =  1952.7
